<a href="https://colab.research.google.com/github/athankefalas/TrafficCongestionSNA/blob/master/ICM%26TwoStep/InfluenceMax_ICM_TwoStep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import networkx as nx
import numpy as np
import random
import itertools as itools

In [0]:
#download graph from drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#make sure to provide the correct path for every graph
file = '/content/drive/My Drive/SNA/Data/toy/toyroad.edges' #google drive path

g = nx.readwrite.read_adjlist(file,comments="%")

In [0]:
#FUNCTION to set activation probabilities
def set_activation_probabilities(G):
  attr={}
  for edge in G.edges():
    attr.update( {edge : 0.1})
                                                 
  nx.set_edge_attributes(G, attr, 'prob')
  
  return G

In [0]:
#FUNCTION for each iteration of diffusion process 
def diffusion_process(Graph,Seed,Visited):
  iter_activations = set()   #this iteration's activations
  iter_visits = set()        #this iteration's visits
  for s in Seed:
    for n in Graph.successors(s):  #successor of n is a node m such that there exists a directed edge from n to m
      if (s, n) in Visited or (s, n) in iter_visits or  n in Seed:   #check if successor was visited before, or seed tried before
        continue
      
      diffusion_prob = g[s][n]['prob']
      activation_prob = np.random.uniform(0.0, 1.0)
      activation = activation_prob >= (1.0-diffusion_prob)
      
      if activation:               
        iter_activations.add(n)

      iter_visits.add((s, n))

  iter_activations = list(iter_activations)
  Seed.extend(iter_activations)
  return Seed, iter_activations, iter_visits

In [0]:
#FUNCTION for ICM and two step ICM 
def ICM(g,start_seeds,num_iter,multi_step=False,step_iter=0):
  visited_edges = set() 
  activated_nodes = []
  num_nodes=[]
  
  
  if multi_step==True:  #split the seed nodes to 2 groups for two step
    
    multi_seeds=start_seeds[len(start_seeds)//2:]
    seeds=start_seeds[0:len(start_seeds)//2]
    
    
  else:
    seeds=start_seeds
      
  len_seeds=len(seeds)
    
  for s in seeds:
    activated_nodes.append(s)
  num_nodes.append(len_seeds)
  
  while True:
    len_seeds=len(seeds)
    
    if multi_step==True and num_iter==step_iter: #activate multi_seeds
      for s in multi_seeds:
        seeds.append(s)
      
    seeds, iter_activations, iter_visits = diffusion_process(g, seeds, visited_edges)
    
    num_nodes.append(len(iter_activations))
    
    for i in iter_activations:
      activated_nodes.append(i)
    visited_edges = visited_edges.union(iter_visits)
    
    if len(seeds) == len_seeds:
      break
    len_seeds=len(seeds)
    num_iter-= 1
    if not num_iter or len(seeds) >= len(g):
      break

  return activated_nodes

In [0]:
def maximize_diffusion(road_net, nodes, budget, iter_count=100):
    candidate_seeds = set()

    # get sets of candidate seeds
    for combination in itools.combinations(nodes, budget):
        seeds = set(combination)

        if len(seeds) != budget:
            continue

        candidate_seeds.add(tuple(seeds))

    max_affected = 0
    optimal_seeds = tuple()

    # find the set of seeds that maximizes the diffusion
    for seeds in candidate_seeds:
        affected_avg = 0.0
        
        for n in range(0, iter_count):
          
          affected = ICM(road_net, list(seeds),num_iter=20)
          affected_avg += len(affected)
          
        affected_avg = affected_avg / iter_count

        if affected_avg >= max_affected:
            max_affected = affected_avg
            optimal_seeds = seeds

    return list(optimal_seeds)

In [0]:
g=set_activation_probabilities(g)

In [0]:
if not g.is_directed():
    g = g.to_directed()

In [0]:
nodes = list(set(nx.nodes(g)))

In [0]:
critical_nodes_count=3

In [0]:
print("Finding best persons to start interactions...")
optimal_max = maximize_diffusion(g, nodes, critical_nodes_count,iter_count=1000)

In [0]:
print("Great spread of information when starting from:", optimal_max)